In [ ]:
#通过交叉验证获得预测值
#函数cross_val_predict和cross_val_score功能相似，但前者对每个输入返回一个预测元素
from sklearn.model_selection import cross_val_predict
predicted=cross_val_predict(clf,iris.data,iris.target,cv=10)
metrics.accuracy_score(iris.target,predicted)

In [ ]:
#生成文件
submission_df=pd.DataFrame(data={'Id':test_df.index,'SalePrice':y_final})
submission_df.head()
submission_df.to_csv('ensenbl_ridge_randomForest.csv',header=False,index=False)

In [ ]:
#模型基础验证
#建立模型SVR
svc=svm.SVR(kernel='rbf',C=10,gamma=0.001)
#训练模型
svc.fit(X_train,y_train)
#得到分数
print(svc.score(X_test,y_test))


#交叉验证
scores = cross_val_score(svc, X_train, y_train,cv=5,scoring='r2')
print("Accuary: %0.2f(+/-%0.2f)" % (scores.mean(),scores.std()*2))

#判断参数


In [ ]:
#随机森林回归调参
N_estimators=[20,50,100,150,200,260,300]
test_scores=[]
for N in N_estimators:
    clf=RandomForestRegressor(n_estimators=N,max_features=0.3)
    test_score=np.sqrt(-cross_val_score(clf,X_train,y_train,cv=5,scoring='neg_mean_squared_error'))
    test_scores.append(np.mean(test_score))
plt.plot(N_estimators,test_scores)
plt.title('N_estimators vc CV Error')

In [ ]:
#保存模型
import joblib

joblib.dump(svc, 'SVR.pickle')
joblib.dump(svcRF, 'svcRF.pickle')
#载入模型
svc = joblib.load('SVR.pickle')
svcRF = joblib.load('svcRF.pickle')

In [ ]:
#练习：字典定义 多变量循环
dic={"test1": [1, 2, 3, 4],"test2": [6, 7, 8, 9],"test3":[10, 11, 12, 13],"test4":[14, 15, 16, 17]}
t1=dic['test1']
t2=dic['test2']
t3=dic['test3']
t4=dic['test4']


for (a,b,c) in zip(dic['test1'],dic['test2'],dic['test3']):
    print("%0.3f(%0.03f) for %r"%(a,b,c))

In [3]:
#xgboost的安装试验代码
import numpy as np  
import xgboost as xgb  
data = np.random.rand(5,10) # 5 entities, each contains 10 features  
label = np.random.randint(2, size=5) # binary target  
dtrain = xgb.DMatrix( data, label=label)  
  
dtest = dtrain  
  
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }  
param['nthread'] = 4  
param['eval_metric'] = 'auc'  
  
evallist  = [(dtest,'eval'), (dtrain,'train')]  
  
num_round = 10  
bst = xgb.train( param, dtrain, num_round, evallist )  
  
bst.dump_model('dump.raw.txt')




[0]	eval-auc:0.5	train-auc:0.5
[1]	eval-auc:0.5	train-auc:0.5
[2]	eval-auc:0.5	train-auc:0.5
[3]	eval-auc:0.5	train-auc:0.5
[4]	eval-auc:0.5	train-auc:0.5
[5]	eval-auc:0.5	train-auc:0.5
[6]	eval-auc:0.5	train-auc:0.5
[7]	eval-auc:0.5	train-auc:0.5
[8]	eval-auc:0.5	train-auc:0.5
[9]	eval-auc:0.5	train-auc:0.5


In [ ]:
#stacking
#定义分类器
clfs = [
        RandomForestClassifier(n_estimators = n_trees, criterion = 'gini'),
        ExtraTreesClassifier(n_estimators = n_trees * 2, criterion = 'gini'),
        GradientBoostingClassifier(n_estimators = n_trees),
    ]
#划分测试机
dev_cutoff = len(Y) * 4/5
    X_dev = X[:dev_cutoff]
    Y_dev = Y[:dev_cutoff]
    X_test = X[dev_cutoff:]
    Y_test = Y[dev_cutoff:]
#定义第二层的训练数据 blend_train
blend_train = np.zeros((X_dev.shape[0], len(clfs))) # Number of training data x Number of classifiers
blend_test = np.zeros((X_test.shape[0], len(clfs))) # Number of testing data x Number of classifiers

#得到blend_train 和 blend_test
for j, clf in enumerate(clfs):
        print 'Training classifier [%s]' % (j)
        blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
        for i, (train_index, cv_index) in enumerate(skf):
            print 'Fold [%s]' % (i)
            
            # This is the training and validation set
            X_train = X_dev[train_index]
            Y_train = Y_dev[train_index]
            X_cv = X_dev[cv_index]
            Y_cv = Y_dev[cv_index]
            
            clf.fit(X_train, Y_train)
            
            # This output will be the basis for our blended classifier to train against,
            # which is also the output of our classifiers
            blend_train[cv_index, j] = clf.predict(X_cv)
            blend_test_j[:, i] = clf.predict(X_test)
        # Take the mean of the predictions of the cross validation set
        blend_test[:, j] = blend_test_j.mean(1)

#训练第二层
bclf = LogisticRegression()
bclf.fit(blend_train, Y_dev)

#得到第二层分数
Y_test_predict = bclf.predict(blend_test)
score = metrics.accuracy_score(Y_test, Y_test_predict)
print 'Accuracy = %s' % (score)

In [124]:
X=[[61, 62, 63, 64, 65, 66, 67, 68, 69, 70],[ 71,72, 73, 74, 75, 76, 77,78, 79, 80],[ 81, 82, 83, 84, 85, 86, 87, 88, 89, 90],
   [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],[ 111, 112, 113,114, 115, 116, 117, 118, 119, 120],[ 121, 122, 123, 124, 125, 126,
    127, 128, 129, 130]]
print(X)
Y=np.arange(31,41)
Y=Y.reshape(len(Y),1)
print(np.shape(Y))

[[61, 62, 63, 64, 65, 66, 67, 68, 69, 70], [71, 72, 73, 74, 75, 76, 77, 78, 79, 80], [81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110], [111, 112, 113, 114, 115, 116, 117, 118, 119, 120], [121, 122, 123, 124, 125, 126, 127, 128, 129, 130]]
(10, 1)


In [126]:
import pandas as pd
id_test=pd.read_csv('id_test.csv').values

In [128]:
print(id_test)

[[    1]
 [    2]
 [    3]
 ...
 [27458]
 [27461]
 [27462]]


In [122]:
np.array(X).mean(1)

array([ 65.5,  75.5,  85.5, 105.5, 115.5, 125.5])

In [119]:
#Kfold 用法 不会因为刷新而变化 返回的是index
from sklearn.model_selection import KFold
kf = KFold(3, True,50)

for train_index, test_index in kf.split(X):
    kfold_X={}
    kfold_Y={}
    kfold_X,kfold_Y=X[train_index],X[test_index]
    print('训练集:{}'.format(train_index))
    print('测试集:{}'.format(test_index))
    print(kfold_X)
    print(kfold_Y)
    print(Y[train_index])
    print(Y[test_index])
    print(type(X[test_index]))
    
    #model.fit()


TypeError: only integer scalar arrays can be converted to a scalar index

In [106]:
# 处理后的原训练集 特征数据：dg_pca_train
# 处理后的原训练集合 得分数据：score_train
# 处理后的测试集合：dg_pca_test
# 测试集序号：id_test

# 在dg_pca_train中继续划分数据
# X_train：0.8比例的原训练集特征 用作训练和验证
# X_test：0.2比例的原训练集特征 用作测试（模型选择）
# y_train：0.8比例的原训练集分数 对应X_train
# y_test：0.2比例的原训练集分数 对应X_test

In [71]:
#定义模型        
clfs = [
        RandomForestClassifier(n_estimators = n_trees, criterion = 'gini'),
        ExtraTreesClassifier(n_estimators = n_trees * 2, criterion = 'gini'),
        GradientBoostingClassifier(n_estimators = n_trees),
    ]
#原始数据
X=dg_pca_train


#建立五折交叉验证的系数集合
kf = KFold(5, True,20)
for train_index, test_index in kf.split(X):
        kfold_X_train = {}
        kfold_X_valid = {}

        # 取数据的标签
        y_train, y_test = train_y[train_index], train_y[test_index]
        # 取数据
        kfold_X_train, kfold_X_valid = train[train_index], train[test_index]



#建立模型SVR
svc=svm.SVR(kernel='rbf',C=10,gamma=0.001)
#训练模型
svc.fit(X_train,y_train)
#得到分数
print(svc.score(X_test,y_test))


#交叉验证
scores = cross_val_score(svc, X_train, y_train,cv=5,scoring='r2')
print("Accuary: %0.2f(+/-%0.2f)" % (scores.mean(),scores.std()*2))

#判断参数

TypeError: 'generator' object is not subscriptable